In [1]:
## Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import logging
tf.get_logger().setLevel(logging.ERROR)

# 1. SeriesGan model
from seriesgan import seriesgan

# 2. Data loading
from data_loading import real_data_loading, sine_data_generation
# 3. Metrics
from metrics.discriminative_metrics import discriminative_score_metrics
from metrics.predictive_metrics import predictive_score_metrics
from metrics.visualization_metrics import visualization

2025-03-30 10:12:28.906972: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-30 10:12:29.939542: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-30 10:12:31.154565: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-30 10:12:31.154615: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-30 10:12:31.176592: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os

In [3]:
stocks = ['AABA']

In [4]:
## Newtork parameters
parameters = dict()

parameters['hidden_dim'] = 'same'
parameters['iterations'] = 2000
parameters['batch_size'] = 128
parameters['num_layer'] = 4

for stock in stocks:
    seq_len = 24
    data_name = 'stock'
    original_data = real_data_loading(data_name, seq_len, stock)
    
    seriesgan_result = seriesgan(original_data, parameters, 'same')
    
    ori_data = pd.read_csv(f'data/{stock}.csv')
    
    
    scaler = MinMaxScaler()
    scaler.fit(ori_data)
    
    rows = len(seriesgan_result)
    
    reconstructed_data = np.zeros((rows, 5))  # Target shape
    count = np.zeros((rows, 1))  # Count occurrences (shape (3661,1) to avoid broadcast issues)

    # Step 5: Reconstruct original shape (3661, 6) by averaging overlapping chunk values
    for i in range(rows):  # Iterate over each chunk
        for j in range(24):  # Iterate over each time step in the chunk
            if i + j < rows:
                reconstructed_data[i + j] += seriesgan_result[i, j]  # Add each feature value
                count[i + j] += 1  # Track how many times each row is summed

    # Step 6: Compute the average where overlaps occur
    reconstructed_data /= count  # Element-wise division to normalize overlapping entries

    # Step 7: Apply inverse transformation using the fitted scaler
    generated_data_restored = scaler.inverse_transform(reconstructed_data)  # Shape: (3661, 6)

    # Convert to DataFrame for visualization
    df_generated = pd.DataFrame(generated_data_restored, columns=["Open", "High", "Low", "Close", "Volume"])
    
    
    
    folder = 'Generated Data'
    
    os.makedirs(folder, exist_ok=True)
    
    df_generated.to_csv(os.path.join(folder, f"{stock}_generated.csv"), index=False)

2025-03-30 10:13:38.669078: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-03-30 10:13:39.045269: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


Start Autoencoder Training for Loss
step: 0/2000, AE_loss: 0.1184
step: 1000/2000, AE_loss: 0.0008
step: 1998/2000, AE_loss: 0.0005
Finish Autoencoder Training for Loss
Start Embedding Network Training
step: 0/2000, AE_loss: 0.3186, AE_D_loss: 0.9274
step: 1000/2000, AE_loss: 0.025, AE_D_loss: 0.4917
step: 1998/2000, AE_loss: 0.0243, AE_D_loss: 0.4864
Finish Embedding Network Training
Start Training with Supervised Loss Only
step: 0/2000, S_loss: 0.6212
step: 1000/2000, S_loss: 0.0108
step: 1999/2000, S_loss: 0.001
Finish Training with Supervised Loss Only
Start Joint Training
step: 0/2000, D_loss: 0.9441, G_loss: 14.8084, G_loss_u: 3.0196, G_loss_s: 0.0012, G_loss_ts: 0.2485, AE_loss: 0.072, AE_D_loss: 0.6815
step: 1000/2000, D_loss: 0.3873, G_loss: 2.9831, G_loss_u: 1.9754, G_loss_s: 0.0005, G_loss_ts: 0.0013, AE_loss: 0.0187, AE_D_loss: 0.8699
step: 1999/2000, D_loss: 0.4779, G_loss: 2.6793, G_loss_u: 1.677, G_loss_s: 0.0003, G_loss_ts: 0.0013, AE_loss: 0.0209, AE_D_loss: 0.9261
Fin

In [6]:
AABA = pd.read_csv("./data/AABA.csv")

In [15]:
AABA["Volume"]

0       24232729
1       20553479
2       12829610
3       29422828
4       16268338
          ...   
3014    10979165
3015     8542802
3016     6345124
3017     7556877
3018     6613070
Name: Volume, Length: 3019, dtype: int64

In [9]:
stock_data = pd.read_csv("./data/stock_data.csv")

In [11]:
stock_data["Open"]

0         49.676899
1         50.178635
2         55.017166
3         55.260582
4         52.140873
           ...     
3680    1207.479980
3681    1205.939941
3682    1214.989990
3683    1207.890015
3684    1196.000000
Name: Open, Length: 3685, dtype: float64

In [16]:
ori_data = np.loadtxt(f'data/AABA.csv', delimiter = ",",skiprows = 1)

In [17]:
ori_data

array([[3.9690000e+01, 4.1220000e+01, 3.8790000e+01, 4.0910000e+01,
        2.4232729e+07],
       [4.1220000e+01, 4.1900000e+01, 4.0770000e+01, 4.0970000e+01,
        2.0553479e+07],
       [4.0930000e+01, 4.1730000e+01, 4.0850000e+01, 4.1530000e+01,
        1.2829610e+07],
       ...,
       [6.9770000e+01, 7.0490000e+01, 6.9690000e+01, 7.0060000e+01,
        6.3451240e+06],
       [7.0120000e+01, 7.0320000e+01, 6.9510000e+01, 6.9820000e+01,
        7.5568770e+06],
       [6.9790000e+01, 7.0130000e+01, 6.9430000e+01, 6.9850000e+01,
        6.6130700e+06]])

In [20]:
ori_data[2913]

array([5.86000e+01, 5.88000e+01, 5.81100e+01, 5.85000e+01, 4.16445e+06])

In [18]:
AABA.iloc[2913]

Open           58.60
High           58.80
Low            58.11
Close          58.50
Volume    4164450.00
Name: 2913, dtype: float64